In [4]:
import yfinance as yf

ticker = yf.download("BTC-EUR", start='2015-01-01', end='2024-10-30')

print(ticker)

[*********************100%***********************]  1 of 1 completed

Price                         Adj Close         Close          High  \
Ticker                          BTC-EUR       BTC-EUR       BTC-EUR   
Date                                                                  
2015-01-01 00:00:00+00:00    260.010742    260.010742    264.909882   
2015-01-02 00:00:00+00:00    262.548553    262.548553    263.177216   
2015-01-03 00:00:00+00:00    234.254517    234.254517    262.646881   
2015-01-04 00:00:00+00:00    221.157715    221.157715    239.378296   
2015-01-05 00:00:00+00:00    229.935501    229.935501    232.746048   
...                                 ...           ...           ...   
2024-10-25 00:00:00+00:00  62971.476562  62971.476562  63552.078125   
2024-10-26 00:00:00+00:00  61702.476562  61702.476562  63418.800781   
2024-10-27 00:00:00+00:00  62047.167969  62047.167969  62327.910156   
2024-10-28 00:00:00+00:00  62920.054688  62920.054688  63195.246094   
2024-10-29 00:00:00+00:00  64624.406250  64624.406250  64932.867188   

Price

In [2]:
ticker.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BTC-EUR,BTC-EUR,BTC-EUR,BTC-EUR,BTC-EUR,BTC-EUR
Date,,,,,,
2024-09-30 00:00:00+00:00,58764.406250,58764.406250,59177.386719,58586.597656,58996.039062,13240139908
2024-10-01 00:00:00+00:00,56865.964844,56865.964844,58764.179688,56454.722656,58763.828125,33325134809
2024-10-02 00:00:00+00:00,54983.457031,54983.457031,57646.929688,54388.777344,56871.445312,45388816903
2024-10-03 00:00:00+00:00,54892.132812,54892.132812,56422.355469,54310.675781,54982.835938,36903348604
2024-10-04 00:00:00+00:00,55060.414062,55060.414062,55699.738281,54243.968750,54891.863281,32719806950


In [7]:
import numpy as np
from bokeh.plotting import figure, output_file, show 

#The file to save the model
output_file("test.html")

#Instantiate figure object
graph = figure(x_axis_type = "datetime", title = "BTC-EUR Closing Prices") 

#Names axes
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = 'Price (EUR)'

#Plotting the line graph
x_axis_coordinates = np.array(ticker.index.values, dtype=np.datetime64)
y_axis_coordinates = ticker.Close
color = 'brown'
legend_label = 'BTC-EUR'
graph.line(x_axis_coordinates, y_axis_coordinates, color=color, legend_label = legend_label)
graph.legend.location = 'top_left'
show(graph)